<a href="https://colab.research.google.com/github/Nazaruk-Anton/Image_Classification_with_Pre-trained_Models/blob/main/2_Pretraining_%26_Comparing_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries, Packages, and Data

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [4]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

--2023-07-02 09:02:49--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261482368 (249M) [application/zip]
Saving to: ‘concrete_data_week3.zip’

concrete_data_week3 100%[===================>] 249.37M  34.8MB/s    in 7.9s    

2023-07-02 09:02:57 (31.6 MB/s) - ‘concrete_data_week3.zip’ saved [261482368/261482368]



In [5]:
!unzip -q concrete_data_week3.zip

## Define Global Constants & Construct ImageDataGenerator Instances

In [6]:
# Define the number of classes and image resize dimensions

num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [7]:
# Create an ImageDataGenerator instance for preprocessing the data

data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

# Create a generator for training data

train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

# Create a generator for validation data

validation_generator = data_generator.flow_from_directory(
    "concrete_data_week3/valid",
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode="categorical",
)

Found 30001 images belonging to 2 classes.
Found 10001 images belonging to 2 classes.


## Build, Compile and Fit Model ResNet50

In [8]:
# Build the model architecture using ResNet50 as the base model

model = Sequential()

model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

model.add(Dense(num_classes, activation='softmax'))

model.layers

94765736/94765736 [==============================] - 0s 0us/step


In [9]:
# Freeze the weights of the ResNet50 base model

model.layers[0].trainable = False

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the number of steps per epoch for training and validation

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

# Fit the model to the training data

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-11-de8d7382ad83>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2


In [10]:
# Save the trained model

model.save('classifier_resnet_model.h5', include_optimizer=False)

##Build, Compile and Fit Model VGG16

In [ ]:
# Build the model architecture using VGG16 as the base model

model_vgg16 = Sequential()

model_vgg16.add(VGG16(include_top=False, pooling="avg", weights="imagenet",))
model_vgg16.add(Dense(num_classes, activation="softmax"))

# Freeze the weights of the VGG16 base model
model_vgg16.layers[0].trainable = False

model_vgg16.summary()

In [ ]:
# Compile the model

model_vgg16.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

# Define the number of steps per epoch for training and validation

num_epochs = 2
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

# Fit the model to the training data

history_vgg16 = model_vgg16.fit_generator(
    train_generator ,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


<ipython-input-8-f6851d96f1ef>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_vgg16 = model_vgg16.fit_generator(


Epoch 1/2
176/301 [================>.............] - ETA: 2:06:08 - loss: 0.1220 - accuracy: 0.9610

In [ ]:
# Save the trained model

model_vgg16.save("classifier_vgg16_model.h5")

## Model Comparison

In [11]:
# Load the saved ResNet50 and VGG16 models

from keras.models import load_model

model_resnet50 = load_model("/content/classifier_resnet_model.h5")
model_vgg16 = load_model("/content/classifier_vgg16_model.h5")


In [18]:
# Create a generator for testing data

testing_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [ ]:
# Evaluate the performance of the VGG16 model

performance_vgg16 = model_vgg16.evaluate_generator(testing_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg16[0], 5)))
print("Accuracy: {}".format(round(performance_vgg16[1], 5)))

<ipython-input-19-23006ffdd2e8>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_vgg16 = model_vgg16.evaluate_generator(testing_generator)


In [ ]:
# Evaluate the performance of the ResNet50 model

performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)
print("Performance of the ResNet50-trained model")
print("Loss: {}".format(round(performance_resnet50[0], 5)))
print("Accuracy: {}".format(round(performance_resnet50[1], 5)))

In [ ]:
# Generate predictions using the VGG16 model

predictions_vgg16 = model_vgg16.predict_generator(testing_generator, steps=1)

# Function to print the prediction result

def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))
    else:
        print("Unsure (prediction split 50–50)")

# Print the first five predictions for the VGG16 model

print("First five predictions for the VGG16-trained model:")
for i in range(5):
    print_prediction(predictions_vgg16[i])

In [ ]:
# Generate predictions using the ResNet50 model

predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)
print("First five predictions for the ResNet50-trained model:")
for i in range(5):
    print_prediction(predictions_resnet50[i])
